In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library(stringr) 

In [ ]:
annoCols<-list(GeneGroup=c(G0='#A6CEE3',
                           G1='#1F78B4',
                           G2='#B2DF8A',
                           G3='#33A02C', 
                           G4='#FB9A99', 
                           G5='#FDBF6F', 
                           G6='#FF7F00', 
                           G7='#CAB2D6',
                           G8='#6A3D9A', 
                           G9='#FFFF99', 
                           G10="#B5651D" ),
                 GuideGroup=c(K0="#1f77b4",
                              K1="#ff7f0e",
                              K2="#279e68",
                              K3="#d62728", 
                              K4="#aa40fc", 
                              K5="#8c564b"))

myGeneModules = data.frame(read.csv("./../TextFiles/ME_GeneModules_leiden_11_Modules.csv",
                                    row.names=1,
                                    stringsAsFactors = FALSE), stringsAsFactors = FALSE)

myGeneModules$GeneName[myGeneModules$GeneName == "X2010111I01Rik"] = "2010111I01Rik"
myGeneModules$GeneName[myGeneModules$GeneName == "X0610012G03Rik"] = "0610012G03Rik"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.M2"] = "H2-M2"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.Q6"] = "H2-Q6"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.Q7"] = "H2-Q7"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.D1"] = "H2-D1"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.K1"] = "H2-K1"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.T23"] = "H2-T23"
myGeneModules$GeneName[myGeneModules$GeneName == "X2010005H15Rik"] = "2010005H15Rik"
myGeneModules$GeneName[myGeneModules$GeneName == "X2310001H17Rik"] = "2310001H17Rik"
myGeneModules$GeneName[myGeneModules$GeneName == "X2810474O19Rik"] = "2810474O19Rik"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.Aa"] = "H2-Aa"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.Ab1"] = "H2-Ab1"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.DMa"] = "H2-DMa"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.DMb1"] = "H2-DMb1"
myGeneModules$GeneName[myGeneModules$GeneName == "H2.Eb1"] = "H2-Eb1"

rownames(myGeneModules) <- myGeneModules$GeneName
myGeneModules$GeneName <- NULL
myGeneModules$GeneGroup <- factor(myGeneModules$GeneGroup)


In [ ]:
doubleRes <- readRDS("./../Notebooks/CombinatorialPerturbations/RDSFiles/ComboEffects_doublesResampleRes.rds")
rownames(doubleRes) = doubleRes$respGene
doubleRes$respGene = NULL
head(doubleRes)

In [ ]:
combosInter <- readRDS("./../Notebooks/CombinatorialPerturbations/RDSFiles/ComboEffects_lm_residuals_withInteractions.rds")
combosInter <- combosInter[combosInter$term %ni% c("(Intercept)", 
                                                   "n_genes", 
                                                   "mt_frac", 
                                                   paste0("leiden", 1:9)),]
combosInter = data.table(combosInter)
combosInter[,FDR:=p.adjust(p.value, 
                           method = "fdr", 
                           n = length(p.value)),
            by=respGene]
combosInter = data.frame(combosInter)



combosInterFDR = copy(combosInter)
combosInter[,c("std.error", "statistic", "p.value","FDR")] = NULL
combosInter = reshape(combosInter, 
                      idvar = "respGene", 
                      timevar = "term", 
                      direction = "wide")
rownames(combosInter) = combosInter$respGene
combosInter$respGene = NULL
colnames(combosInter) = sapply( colnames(combosInter), 
                               function(x){strsplit(x,"estimate.")[[1]][2]})

combosInterCopy = copy(combosInter)


combosInterFDR[,c("std.error", "statistic", "estimate", "p.value")] = NULL
combosInterFDR = reshape(combosInterFDR, 
                         idvar = "respGene", 
                         timevar = "term",  
                         direction = "wide")
rownames(combosInterFDR) = combosInterFDR$respGene
combosInterFDR$respGene = NULL
colnames(combosInterFDR) = sapply( colnames(combosInterFDR), 
                                  function(x){strsplit(x,"FDR.")[[1]][2]})


combosInter[combosInterFDR > 0.1] = 0

In [ ]:
cMean1 = data.frame(readRDS("./../Notebooks/CombinatorialPerturbations/RDSFiles/ControlCellsMeanExp.rds"), stringsAsFactors = FALSE)
rownames(cMean1) = cMean1$Genes
cMean1$Genes = NULL
cMean1 = data.frame(scale(cMean1, 
                          center = TRUE, 
                          scale = FALSE), 
                    stringsAsFactors = FALSE)

write.csv(combosInter, "CombosInter.csv")

In [ ]:
doubleRes = doubleRes[rownames(combosInter),]
combosInter = cbind(combosInter, doubleRes)
combosInterPlot = copy(combosInter)
colnames(combosInterPlot) <- sapply(colnames(combosInterPlot), function(x){str_replace_all(x,"_","")})

combosInterPlot = combosInterPlot[rownames(myGeneModules),]
combosInterPlot[combosInterPlot > 0.2] =0.2
combosInterPlot[combosInterPlot <  -0.2] = -0.2

combosInterPlot <- t(combosInterPlot)
initialRowNames = rownames(combosInterPlot)


In [ ]:
options(repr.plot.width=20, repr.plot.height=8)

allGenesInorder = c()
for (elem in unique(myGeneModules$GeneGroup)){
    aa = combosInterPlot[,rownames(myGeneModules[myGeneModules$GeneGroup == elem, ])]
    k = pheatmap(aa,
             cluster_rows = FALSE,
             colorRampPalette(c("blue", "white", "orange"))(100),
             clustering_method="ward.D2")
    
    allGenesInorder = c(allGenesInorder, colnames(aa)[k$tree_col$order])

}


In [ ]:
km2 <- pheatmap(combosInterPlot[,allGenesInorder], 
         cluster_cols = FALSE,
         cluster_rows = TRUE,
         clustering_method="ward.D",
         treeheight_col=0,fontsize_col=3,fontsize_row=18,
         colorRampPalette(c("navy", "white", "red"))(50))


In [ ]:

myGeneOrder = allGenesInorder
myrowOrder = rownames(combosInterPlot)[km2$tree_row$order]

cMean1 = cMean1[myGeneOrder,]
combosInterPlot = data.frame(rbind(combosInterPlot,cMean1$MeanExp_Cont2))


combosInterPlot = combosInterPlot[1:27,]
rownames(combosInterPlot) = c(initialRowNames, "AverageControlExpression")

myAnno = myGeneModules[myGeneOrder,]
myAnno$GeneColor = NULL

names(combosInterPlot)[names(combosInterPlot) == "X2010111I01Rik"] = "2010111I01Rik"
names(combosInterPlot)[names(combosInterPlot) == "X0610012G03Rik"] = "0610012G03Rik"
names(combosInterPlot)[names(combosInterPlot) == "H2.M2"] = "H2-M2"
names(combosInterPlot)[names(combosInterPlot) == "H2.Q6"] = "H2-Q6"
names(combosInterPlot)[names(combosInterPlot) == "H2.Q7"] = "H2-Q7"
names(combosInterPlot)[names(combosInterPlot) == "H2.D1"] = "H2-D1"
names(combosInterPlot)[names(combosInterPlot) == "H2.K1"] = "H2-K1"
names(combosInterPlot)[names(combosInterPlot) == "H2.T23"] = "H2-T23"
names(combosInterPlot)[names(combosInterPlot) == "X2010005H15Rik"] = "2010005H15Rik"
names(combosInterPlot)[names(combosInterPlot) == "X2310001H17Rik"] = "2310001H17Rik"
names(combosInterPlot)[names(combosInterPlot) == "X2810474O19Rik"] = "2810474O19Rik"
names(combosInterPlot)[names(combosInterPlot) == "H2.Aa"] = "H2-Aa"
names(combosInterPlot)[names(combosInterPlot) == "H2.Ab1"] = "H2-Ab1"
names(combosInterPlot)[names(combosInterPlot) == "H2.DMa"] = "H2-DMa"
names(combosInterPlot)[names(combosInterPlot) == "H2.DMb1"] = "H2-DMb1"
names(combosInterPlot)[names(combosInterPlot) == "H2.Eb1"] = "H2-Eb1"

xx <- pheatmap(combosInterPlot[c(myrowOrder,"AverageControlExpression"), myGeneOrder], 
         cluster_cols = FALSE,
         cluster_rows = FALSE,
         clustering_method="ward.D",
         treeheight_col=0,fontsize_col=3,fontsize_row=18,
         annotation_col = myAnno,
         annotation_colors = annoCols,
         colorRampPalette(c("navy", "white", "red"))(50))

save_pheatmap_pdf(xx, "Figure_5C.pdf", width=20, height=8)

In [ ]:
write.csv(combosInterPlot[c(myrowOrder,"AverageControlExpression"), myGeneOrder], "Fig_5C.csv")

In [ ]:
myInsignGenes = sapply(combosInter, function(x){length(which(x == 0))})

kk = data.frame(GroupNames=names(myInsignGenes), Insignificant = myInsignGenes)
kk$Significant = 1041 - kk$Insignificant
kk = melt(kk, id.vars="GroupNames")
kk$GroupNames = factor(kk$GroupNames, levels = colnames(combosInter))
kk$GroupNames <- sapply(kk$GroupNames, function(x){str_replace_all(x,"_","")})

In [ ]:
options(repr.plot.width=10, repr.plot.height=7)

#pdf("./../SupplementaryFigures/S_6C.pdf", width=12, height=6)
ggplot(kk, aes(fill=variable, y=value, x=GroupNames)) + 
    geom_bar(position="stack", stat="identity")+
    scale_fill_manual("Effect size",values=c("grey","blue"))+
    ylab("Number of tested genes")+theme_minimal()+
     theme(axis.text.y = element_text(size = 15),
           axis.text.x = element_text(size = 15, angle=90),
              axis.title = element_text(size = 20),
              strip.text = element_text(size = 25),
              legend.text=element_text(size=20),
              legend.title=element_text(size=20),
              legend.position="top")+xlab("KO groups and their interactions")
#dev.off()

In [ ]:
combosInteractionsOnly = combosInter[,colnames(combosInter) %ni% 
                                     c("K_0", "K_1", "K_2", "K_3", "K_4", "K_5")]

combosInteractionsOnly_interMod = combosInteractionsOnly[,colnames(combosInteractionsOnly)[colnames(combosInteractionsOnly) %ni% c("K0:K0", 
                                                                                                                                   "K1:K1",
                                                                                                                                   "K2:K2",
                                                                                                                                   "K3:K3",
                                                                                                                                   "K4:K4")]]
combosInteractionsOnly_intraMod = combosInteractionsOnly[,c("K0:K0", "K1:K1","K2:K2","K3:K3","K4:K4")]

sgnInterModInteractions = apply(combosInteractionsOnly_interMod,1, 
                                function(x){length(which(x != 0))})
sgnIntraModInteractions = apply(combosInteractionsOnly_intraMod,1, 
                                function(x){length(which(x != 0))})

kk = data.frame(GroupNames=names(sgnInterModInteractions), 
                InterModule_Significant = sgnInterModInteractions,
                IntraModule_Significant = sgnIntraModInteractions)
kk$InSignificant = 20 - (sgnInterModInteractions + sgnIntraModInteractions)
kk = kk[order(-kk$InSignificant, -kk$IntraModule_Significant, -kk$InterModule_Significant),]
kk = melt(kk, id.vars="GroupNames")
kk$GroupNames = factor(kk$GroupNames, levels=kk$GroupNames[1:1041])
kk

Suplementary Figure 9F

In [ ]:
options(repr.plot.width=15, repr.plot.height=7)

pdf("Figure_S6B.pdf", width=20, height=8)
ggplot(kk, aes(fill=variable, y=value, x=GroupNames)) + 
    geom_bar(position="stack", stat="identity")+
    scale_fill_manual("Effect size",values=c("orange", "red", "grey"))+
    ylab("Number of interaction terms")+theme_minimal()+
     theme(axis.text.y = element_text(size = 15),
           axis.text.x = element_text(size = 2, angle=90),
              axis.title = element_text(size = 20),
              strip.text = element_text(size = 25),
              legend.text=element_text(size=20),
              legend.title=element_text(size=20),
              legend.position="top")+xlab("Genes")

dev.off()

In [ ]:
combosInterMyTemp = combosInter[,colnames(combosInter) %ni% c("K_0", "K_1", "K_2", "K_3", "K_4", "K_5")]
combosInterMyTemp$Gene =rownames(combosInterMyTemp)
combosInterMelt = melt(combosInterMyTemp, id.vars="Gene")
combosInterMelt = combosInterMelt[combosInterMelt$value != 0,]
combosInterMelt$Group1 = sapply(combosInterMelt$variable, 
                                function(x){strsplit(as.character(x),":")[[1]][1]})
combosInterMelt$Group2 = sapply(combosInterMelt$variable, 
                                function(x){strsplit(as.character(x),":")[[1]][2]})
combosInterMelt$GeneGroup1 = paste0(combosInterMelt$Gene,"_",combosInterMelt$Group1)
combosInterMelt$GeneGroup2 = paste0(combosInterMelt$Gene,"_",combosInterMelt$Group2)

In [ ]:
combosInterMyTemp_2 = combosInter[,colnames(combosInter) %in% c("K_0", "K_1", "K_2", "K_3", "K_4", "K_5")]
combosInterMyTemp_2$Gene =rownames(combosInterMyTemp_2)
combosInterMelt_2 = melt(combosInterMyTemp_2, id.vars="Gene")
combosInterMelt_2$GeneGroup = paste0(combosInterMelt_2$Gene,"_",combosInterMelt_2$variable)
combosInterMelt_2 = combosInterMelt_2[,c("GeneGroup", "value")]

combosInterMelt_21 = copy(combosInterMelt_2)
colnames(combosInterMelt_21) = c("GeneGroup1", 
                                 "Group1Value")
combosInterMelt_22 = copy(combosInterMelt_2)
colnames(combosInterMelt_22) = c("GeneGroup2", 
                                 "Group2Value")

head(combosInterMelt_21)

In [ ]:
combosInterMelt = merge(combosInterMelt, combosInterMelt_21, by="GeneGroup1")
combosInterMelt = merge(combosInterMelt, combosInterMelt_22, by="GeneGroup2")
combosInterMelt = combosInterMelt[order(combosInterMelt$Group1, combosInterMelt$Group2, 
                                        combosInterMelt$Group1Value, combosInterMelt$Group2Value, combosInterMelt$value),]

In [ ]:
options(repr.plot.width=4, repr.plot.height=8)

for(elem in unique(combosInterMelt$variable)){
    
    combosInterMeltGroup = combosInterMelt[combosInterMelt$variable == elem,]
    
    g1 = unique(combosInterMeltGroup$Group1)
    g2 = unique(combosInterMeltGroup$Group2)
    
    tp = combosInterMeltGroup[,c("Group1Value", "Group2Value","value")]
    tp$g1Ag2 = rowSums(tp)
    tp$g1g2 = tp$Group1Value + tp$Group2Value
    colnames(tp) = c(g1, g2,elem, paste0(g1,g2), paste0(g1,"+",g2))
    tp = tp[,c(g1, g2,paste0(g1,"+",g2),elem, paste0(g1,g2))]
    rownames(tp) = combosInterMeltGroup$Gene
    
    myAnnot = myGeneModules[rownames(tp),]
    myAnnot$GeneColor = NULL 
    
    #print(tp)

    tp[tp > 0] =1
    tp[tp <  0] = -1
    tp$GeneGroup = myAnnot
    tp = tp[order(-tp[,1], -tp[,2], -tp[,3], -tp[,4],-tp[,5],tp$GeneGroup),]
    tp$GeneGroup = NULL
   
    colnames(tp) <- sapply(colnames(tp), function(x){str_replace_all(x,"_","")})
    
    tt <- pheatmap(tp,
         cluster_cols = FALSE,
         cluster_rows = FALSE,
         clustering_method="ward.D",
         treeheight_col=0,
         treeheight_row=0,
         fontsize_row=8,fontsize_col=15,
         annotation_row = myAnnot,
         annotation_colors = annoCols,
         colorRampPalette(c("navy", "white", "red"))(50))
    
    save_pheatmap_pdf(tt, paste0(elem,".pdf"), width=4, height=15)

}
